## Let us consider two markovian Gaussian Process. We will find the adapted Wasserstein distance between them using our typical solver and a markovian solver and compare perofrmance

In [5]:
import os, sys, numpy as np, time, random, concurrent.futures
import matplotlib.pyplot as plt

# Set up source paths (adjust as necessary)
notebooks_path = os.path.abspath(os.getcwd())
src_path = os.path.abspath(os.path.join(notebooks_path, "../src"))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# Import your custom functions (including markovian ones) from files.
from optimal_code.utils import *
from optimal_code.optimal_solver import *
from optimal_code.optimal_solver_markov import *

from trees.build_trees_from_paths import *
from trees.treeVisualization import *
from adapted_empirical_measure.AEM_grid import *


from benchmark_code.aot_numerics.mainfunctions import *
from benchmark_code.aot_numerics.measure import *
from benchmark_code.aot_numerics.FVI_bench import *
from benchmark_code.aot_numerics.aot_measure_to_path import *

In [10]:
N_INSTANCE = 1
def cost(x, y):
    return (x[0] - y[0]) ** 2

# Prepare storage for repeated runs
final_result_bw = np.zeros(N_INSTANCE)
final_result_pot = np.zeros(N_INSTANCE)
bw_times = []
pot_times = []

T_val = 12

# Build the graph for the Markov chain
g = Graph(T_val + 1)
for t in range(T_val):
    g.addEdge(t, t + 1)

# Start measuring time for all N_INSTANCE runs (optional, not necessarily used below)
overall_t0 = time()

for n_ins in range(N_INSTANCE):
    print(f"\n  Instance {n_ins+1}/{N_INSTANCE} for T={T_val}")

    # Generate random measures mu, nu
    mu, supp_mu = rand_tree_binom(
        T_val, init=0, vol=1, N_leaf=2, in_size=30
    )
    nu, supp_nu = rand_tree_binom(
        T_val, init=0, vol=2, N_leaf=2, in_size=30
    )

    x_init = 0
    y_init = 0
    # Extract sample paths
    sample_path_x, weight_x = extract_sample_paths(mu, T_val, x_init)
    sample_path_y, weight_y = extract_sample_paths(nu, T_val, y_init)

    sample_path_x = sample_path_x.T
    sample_path_y = sample_path_y.T


    q2v = np.unique(np.concatenate([sample_path_x, sample_path_y], axis=0))
    v2q = {k: v for v, k in enumerate(q2v)}
    qX = np.array([[v2q[x] for x in y] for y in sample_path_x])
    qY = np.array([[v2q[x] for x in y] for y in sample_path_y])
    # Transpose so that each row corresponds to one sample path.
    qX = sort_qpath(qX.T)
    qY = sort_qpath(qY.T)
    
    # ---------------------------
    # Non-Markovian solver
    # ---------------------------
    mu_x = qpath2mu_x(qX, markovian=True)  # Non-Markovian conditional measure.
    nu_y = qpath2mu_x(qY, markovian=True)
    (mu_x_c, mu_x_cn, mu_x_v, mu_x_w, mu_x_cumn, v2q_x, mu_x_idx) = \
        list_repr_mu_x_markovian(mu_x, q2v)
    (nu_y_c, nu_y_cn, nu_y_v, nu_y_w, nu_y_cumn, v2q_y, nu_y_idx) = \
        list_repr_mu_x_markovian(nu_y, q2v)
    

    # ----------------
    # 1) POT Solver
    # ----------------
    print("    Computing adapted OT using POT solver...")
    start_time_pot = time()
    distance_pot = nested2_parallel_markovian(mu_x_cn, mu_x_v, mu_x_w, mu_x_idx,
                                                nu_y_cn, nu_y_v, nu_y_w, nu_y_idx,
                                                n_processes=42)
    elapsed_time_pot = time() - start_time_pot
    pot_times.append(elapsed_time_pot)

    # Subtract (x_init - y_init)^2 to be consistent with your example
    distance_pot_adjusted = distance_pot - (x_init - y_init) ** 2
    final_result_pot[n_ins] = distance_pot_adjusted

    print(
        f"    POT Distance: {distance_pot_adjusted:.4f}  Time: {elapsed_time_pot:.2f}s"
    )

    # ----------------
    # 2) Backward Induction (BW)
    # ----------------
    cost_funs = [[[t], cost] for t in range(T_val + 1)]
    start_time_bw = time()
    BW_v1, _ = solve_dynamic(
        cost_funs, mu, nu, supp_mu, supp_nu, g, outputflag=0, method="pot"
    )
    elapsed_time_bw = time() - start_time_bw
    bw_times.append(elapsed_time_bw)

    # Also subtract (x_init - y_init)^2
    BW_v1_adjusted = BW_v1[0] - (x_init - y_init) ** 2
    final_result_bw[n_ins] = BW_v1_adjusted

    print(f"    BW Distance: {BW_v1_adjusted:.4f}  Time: {elapsed_time_bw:.2f}s")

# Summarize the results for T_val
mean_bw = np.mean(final_result_bw)
std_bw = np.std(final_result_bw)
mean_pot = np.mean(final_result_pot)
std_pot = np.std(final_result_pot)

mean_bw_time = np.mean(bw_times)
mean_pot_time = np.mean(pot_times)

overall_elapsed = time() - overall_t0

print("\n  Summary for T =", T_val)
print("  BW mean (std)  =", f"{mean_bw:.4f} ({std_bw:.4f})")
print("  POT mean (std) =", f"{mean_pot:.4f} ({std_pot:.4f})")
print("  BW avg time    =", f"{mean_bw_time:.3f}")
print("  POT avg time   =", f"{mean_pot_time:.3f}")
print("  Overall time   =", f"{overall_elapsed:.3f}s")


  Instance 1/1 for T=12
    Computing adapted OT using POT solver...
    POT Distance: 90.6845  Time: 20.36s
    BW Distance: 90.6845  Time: 686.96s

  Summary for T = 12
  BW mean (std)  = 90.6845 (0.0000)
  POT mean (std) = 90.6845 (0.0000)
  BW avg time    = 686.961
  POT avg time   = 20.362
  Overall time   = 744.546s


In [13]:
N_INSTANCE = 1
def cost(x, y):
    return (x[0] - y[0]) ** 2

# Prepare storage for repeated runs
final_result_bw = np.zeros(N_INSTANCE)
final_result_pot = np.zeros(N_INSTANCE)
bw_times = []
pot_times = []

T_val = 13

# Build the graph for the Markov chain
g = Graph(T_val + 1)
for t in range(T_val):
    g.addEdge(t, t + 1)

# Start measuring time for all N_INSTANCE runs (optional, not necessarily used below)
overall_t0 = time()

for n_ins in range(N_INSTANCE):
    print(f"\n  Instance {n_ins+1}/{N_INSTANCE} for T={T_val}")

    # Generate random measures mu, nu
    mu, supp_mu = rand_tree_binom(
        T_val, init=0, vol=1, N_leaf=2, in_size=30
    )
    nu, supp_nu = rand_tree_binom(
        T_val, init=0, vol=2, N_leaf=2, in_size=30
    )

    x_init = 0
    y_init = 0
    # Extract sample paths
    sample_path_x, weight_x = extract_sample_paths(mu, T_val, x_init)
    sample_path_y, weight_y = extract_sample_paths(nu, T_val, y_init)

    sample_path_x = sample_path_x.T
    sample_path_y = sample_path_y.T


    q2v = np.unique(np.concatenate([sample_path_x, sample_path_y], axis=0))
    v2q = {k: v for v, k in enumerate(q2v)}
    qX = np.array([[v2q[x] for x in y] for y in sample_path_x])
    qY = np.array([[v2q[x] for x in y] for y in sample_path_y])
    # Transpose so that each row corresponds to one sample path.
    qX = sort_qpath(qX.T)
    qY = sort_qpath(qY.T)

    
    # ---------------------------
    # Non-Markovian solver
    # ---------------------------
    mu_x = qpath2mu_x(qX, markovian=True)  # Non-Markovian conditional measure.
    nu_y = qpath2mu_x(qY, markovian=True)
    (mu_x_c, mu_x_cn, mu_x_v, mu_x_w, mu_x_cumn, v2q_x, mu_x_idx) = \
        list_repr_mu_x_markovian(mu_x, q2v)
    (nu_y_c, nu_y_cn, nu_y_v, nu_y_w, nu_y_cumn, v2q_y, nu_y_idx) = \
        list_repr_mu_x_markovian(nu_y, q2v)
    

    # ----------------
    # 1) POT Solver
    # ----------------
    print("    Computing adapted OT using POT solver...")
    start_time_pot = time()
    distance_pot = nested2_parallel_markovian(mu_x_cn, mu_x_v, mu_x_w, mu_x_idx,
                                                nu_y_cn, nu_y_v, nu_y_w, nu_y_idx,
                                                n_processes=42)
    elapsed_time_pot = time() - start_time_pot
    pot_times.append(elapsed_time_pot)

    # Subtract (x_init - y_init)^2 to be consistent with your example
    distance_pot_adjusted = distance_pot - (x_init - y_init) ** 2
    final_result_pot[n_ins] = distance_pot_adjusted

    print(
        f"    POT Distance: {distance_pot_adjusted:.4f}  Time: {elapsed_time_pot:.2f}s"
    )

    # ----------------
    # 2) Backward Induction (BW)
    # ----------------
    cost_funs = [[[t], cost] for t in range(T_val + 1)]
    start_time_bw = time()
    BW_v1, _ = solve_dynamic(
        cost_funs, mu, nu, supp_mu, supp_nu, g, outputflag=0, method="pot"
    )
    elapsed_time_bw = time() - start_time_bw
    bw_times.append(elapsed_time_bw)

    # Also subtract (x_init - y_init)^2
    BW_v1_adjusted = BW_v1[0] - (x_init - y_init) ** 2
    final_result_bw[n_ins] = BW_v1_adjusted

    print(f"    BW Distance: {BW_v1_adjusted:.4f}  Time: {elapsed_time_bw:.2f}s")

# Summarize the results for T_val
mean_bw = np.mean(final_result_bw)
std_bw = np.std(final_result_bw)
mean_pot = np.mean(final_result_pot)
std_pot = np.std(final_result_pot)
    
mean_bw_time = np.mean(bw_times)
mean_pot_time = np.mean(pot_times)

overall_elapsed = time() - overall_t0

print("\n  Summary for T =", T_val)
print("  BW mean (std)  =", f"{mean_bw:.4f} ({std_bw:.4f})")
print("  POT mean (std) =", f"{mean_pot:.4f} ({std_pot:.4f})")
print("  BW avg time    =", f"{mean_bw_time:.3f}")
print("  POT avg time   =", f"{mean_pot_time:.3f}")
print("  Overall time   =", f"{overall_elapsed:.3f}s")


  Instance 1/1 for T=13
    Computing adapted OT using POT solver...
    POT Distance: 64.3860  Time: 88.40s
    BW Distance: 64.3844  Time: 2809.10s

  Summary for T = 13
  BW mean (std)  = 64.3844 (0.0000)
  POT mean (std) = 64.3860 (0.0000)
  BW avg time    = 2809.100
  POT avg time   = 88.397
  Overall time   = 2964.362s
